In [ ]:
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
import cv2 
import os
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, io, models
import os
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def display_images(imges):
  fig = plt.figure(figsize=(10, 5))
  rows = 1
  columns = 10
  for i,img in enumerate(imges):
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img)
  plt.show(block = True)

In [ ]:
class AlDataset(torch.utils.data.Dataset):

  def __init__(self, train_dir, transforms = None):
    super().__init__()
    self.train_dir = train_dir
    self.transforms = transforms
    self.train_images = []
    self.read_data()

   

  def __len__(self):
    return len(self.train_images)

  def __getitem__(self, idx):
    return {'image': self.transforms(io.read_image(self.train_images[idx]['image']).repeat(3,1,1)) , 'label': self.train_images[idx]['label']}
  
  def read_data(self):

      self.labels = os.listdir(self.train_dir)

      for i, subdir in enumerate(self.labels):
        images = os.listdir(self.train_dir + '/' + subdir)
        for img in images:
          data = {}
          data['image'] = self.train_dir + '/' + subdir + '/' + img
          data['label'] = i
          self.train_images.append(data)
        
      
      random.shuffle(self.train_images)




commented this piece of code as it was used to generate the mean & standard deviation of the dataset for normalizing the dataset

In [ ]:
# psum    = torch.tensor([0.0, 0.0, 0.0])
# psum_sq = torch.tensor([0.0, 0.0, 0.0])


# for i, batch in enumerate(trainLoader): 
#     _batch, labels = batch['image'], batch['label']
    
#     psum    += _batch.sum(axis = [ 0, 2, 3])
#     psum_sq += (_batch ** 2).sum(axis = [0, 2, 3])
    
# count = trainDataset.__len__() * 208 * 176

# # mean and std
# total_mean = psum / count
# total_var  = (psum_sq / count) - (total_mean ** 2)
# total_std  = torch.sqrt(total_var)

# # output
# print('mean: '  + str(total_mean))
# print('std:  '  + str(total_std))

In [ ]:
trainDataset = AlDataset('/kaggle/input/alzeihmer/Alzheimer_s Dataset/train', transforms = transforms.Compose([
                                    transforms.ConvertImageDtype(torch.float), 
                                    transforms.
                                    transforms.Normalize([0.2821, 0.2821,0.2821], [0.3258, 0.3258, 0.3258])
                                        ]))
testDataset = AlDataset('/kaggle/input/alzeihmer/Alzheimer_s Dataset/test' , transforms = transforms.Compose([
                                    transforms.ConvertImageDtype(torch.float),
                                    transforms.Normalize([0.2821], [0.3258])
                                        ]))

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
trainLoader = DataLoader(dataset = trainDataset, batch_size = 256, num_workers = 2, shuffle = True)
testLoader = DataLoader(dataset = testDataset, batch_size = 128)

In [ ]:
class AlNet(nn.Module):
    def __init__(self, num_classes=4):
      super().__init__()
      self.net = nn.Sequential(
          
          #initial 208 X 176 X 1
          
          nn.Conv2d(in_channels = 3, out_channels = 48 , kernel_size = 11, stride = 2),  # 99 X 83 X 48
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),                                         # 49 X 41 X 48
          nn.BatchNorm2d(48),
          
          nn.Conv2d(in_channels=48, out_channels=128, kernel_size=5, padding=1),         # 47 X 39 X 128
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,stride=2),                                          # 23 X 19 X 128
          nn.BatchNorm2d(128)
      )

      self.fc = nn.Sequential(
          nn.Linear(in_features=(23 * 19 * 128), out_features=1024),
          nn.ReLU(),
          nn.Dropout(p=0.5,),
          nn.Linear(in_features=1024, out_features=512),
          nn.ReLU(),
          nn.Linear(in_features=512, out_features=num_classes),
      )
    
      self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
      input = self.net(input)
      input = torch.flatten(input, start_dim=1)
      input = self.fc(input)
      input = self.softmax(input)
      return input


In [ ]:
import torch.optim as optim

alnet = AlNet().to(device)

optimizer = optim.Adam(params=alnet.parameters(), lr=0.0001)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

 # multiply LR by 1 / 10 after every 20 epochs
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
num_classes = 4
learning_rate = 0.001
num_epochs = 25

In [ ]:
print('Starting Training ....')
total_count = 0
total_correct = 0

alnet.train()
for epoch in range(num_epochs):
  lr_scheduler.step()
  for i, batch in enumerate(trainLoader): 
        
      batch, labels = batch['image'].to(device), batch['label'].to(device)

      output = alnet(batch)
      loss = criterion(output, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      total_count += len(batch)
      _, preds = torch.max(output, 1)
      total_correct += torch.sum(preds == labels).sum().item()



  print('Epoch %d/%d Accuracy %0.4f ' % ( epoch, num_epochs, (total_correct/total_count)))

In [ ]:
print('Starting Testing...')
total_count = 0
total_correct = 0

alnet.eval()

for i, batch in enumerate(testLoader): 
    batch, labels = batch['image'].to(device), batch['label'].to(device)

    output = alnet(batch)

    total_count += len(batch)
    _, preds = torch.max(output, 1)
    total_correct += torch.sum(preds == labels).sum().item()

    print(' Accuracy %0.4f ' % ((total_correct/total_count)))

# Transfer Learning #


In [ ]:
class MyMaxPool(nn.Module):
    
    def __init__(self):
        super(MyMaxPool, self).__init__()
        
    def forward(self, x):
        return x

In [ ]:
from torchvision import models


model =  models.vgg16(pretrained=False)

model.avgpool = MyMaxPool()
model.classifier = nn.Sequential(nn.Linear(15360, 4096), 
                                 nn.ReLU(), 
                                 nn.Dropout(0.5),
                                 nn.Linear(4096, 4096),
                                 nn.ReLU(), 
                                 nn.Dropout(0.5),
                                 nn.Linear(4096, 4)
                                )

In [ ]:
import torch.optim as optim


optimizer = optim.Adam(params=model.parameters(), lr=0.0001)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

 # multiply LR by 1 / 10 after every 20 epochs
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

for param in model.parameters:
    

In [ ]:
print('Starting Training ....')
total_count = 0
total_correct = 0
model.to(device)
model.train()
for epoch in range(num_epochs):
    for i, batch in enumerate(trainLoader): 
        
        batch, labels = batch['image'].to(device), batch['label'].to(device)
        output = model(batch)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_count += len(batch)
        _, preds = torch.max(output, 1)
        total_correct += torch.sum(preds == labels).sum().item()
    print('Epoch %d/%d Accuracy %0.4f ' % ( epoch, num_epochs, (total_correct/total_count)))